## Imports ##

In [2]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv("../data/1-raw/openpowerlifting-2025-09-27-31932eca.csv")

print(df.columns)
print(len(df.index))

/var/folders/wm/mz0f2pm95tn34tffq_n21nv80000gn/T/ipykernel_61069/1413922412.py:1: DtypeWarning: Columns (31,33,35,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/1-raw/openpowerlifting-2025-09-27-31932eca.csv")


Index(['Name', 'Sex', 'Event', 'Equipment', 'Age', 'AgeClass',
       'BirthYearClass', 'Division', 'BodyweightKg', 'WeightClassKg',
       'Squat1Kg', 'Squat2Kg', 'Squat3Kg', 'Squat4Kg', 'Best3SquatKg',
       'Bench1Kg', 'Bench2Kg', 'Bench3Kg', 'Bench4Kg', 'Best3BenchKg',
       'Deadlift1Kg', 'Deadlift2Kg', 'Deadlift3Kg', 'Deadlift4Kg',
       'Best3DeadliftKg', 'TotalKg', 'Place', 'Dots', 'Wilks', 'Glossbrenner',
       'Goodlift', 'Tested', 'Country', 'State', 'Federation',
       'ParentFederation', 'Date', 'MeetCountry', 'MeetState', 'MeetTown',
       'MeetName', 'Sanctioned'],
      dtype='object')
3673663


In [ ]:
# Checking unique divisions to understand the data to only keep Open
print("Unique divisions in IPF Raw SBD:")
print(df[
    (df['ParentFederation'] == 'IPF') & 
    (df['Equipment'] == 'Raw') & 
    (df['Event'] == 'SBD')
]['Division'].value_counts().head(20))

Unique divisions in IPF Raw SBD:
Division
Open             157022
MR-O              57626
Juniors           57169
FR-O              33715
Sub-Juniors       30064
MR-Jr             19315
Masters 1         16817
Masters 2         10508
M-C-Open           8415
Seniors            7957
Junior             7872
FR-Jr              7494
Masters 3          5335
F-C-Open           4584
MR-T3              4511
MR-C               3777
MR-T2              3437
MR-Sj              3333
Juniors 19-23      3174
M-O                3143
Name: count, dtype: int64


In [ ]:
# Event: only SBD
# Equipment: Raw only
# Age: drop empty fields
# Division: Open only (catches all with 'O', excludes lowercase 'o' such as 'Juniors')
# TotalKg: drop empty fields (no DQs)
# Place: must be a number (no DQs)
# ParentFederation: only IPF
# Best3: ensure all 3 lifts are successful (no DQs)

data = df[
    (df['Event'] == 'SBD') & 
    (df['Equipment'] == 'Raw') &
    (df['Age'].notna()) &
    (df['Division'].str.contains('O', na=False)) &
    (df['TotalKg'].notna()) &
    (df['Place'].str.isnumeric()) &
    (df['ParentFederation'] == 'IPF') &
    (df['Best3SquatKg'].notna()) &
    (df['Best3BenchKg'].notna()) &
    (df['Best3DeadliftKg'].notna()) 
].copy()

# Convert to datetime
data['Date'] = pd.to_datetime(data['Date'])

# Sort by lifter and date (important for feature engineering)
data = data.sort_values(['Name', 'Date']).reset_index(drop=True)

data.to_csv('../data/2-preprocessed/cleanIPF.csv', index=False)

print(f"Original rows: {len(df)}")
print(f"Filtered rows: {len(data)}")
print(f"Unique lifters: {data['Name'].nunique()}")
print(f"Male: {len(data[data['Sex']=='M'])}, Female: {len(data[data['Sex']=='F'])}")

/var/folders/wm/mz0f2pm95tn34tffq_n21nv80000gn/T/ipykernel_61069/912407530.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Date'] = pd.to_datetime(data['Date'])


Original rows: 3673663
Filtered rows: 208785
Unique lifters: 94360
Male: 136197, Female: 72585
